## Select the analysis mode from the dropdown menu below.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from measurement_directory import *
import analysis_wrappers

def set_analysis_type(analysis_mode):
    global analysis_type
    analysis_type = analysis_mode
    print('analysis mode is {a_type} \n'.format(a_type = analysis_type))

mode_select_widget = interact(set_analysis_type,analysis_mode=['fake analysis' , 'fake analysis 2', 'ycam', 'zcam_dual_imaging'])

no matlab engine, debugging mode


interactive(children=(Dropdown(description='analysis_mode', options=('fake analysis', 'fake analysis 2', 'ycam…

## Enter the run from today you wish to analyze.

In [3]:
global watchfile
print('today\'s measurement(s):')
print(todays_measurements())
measurement_dir = measurement_directory()
watchfile = measurement_dir + r'\image_log.csv'

today's measurement(s):
['foo', 'foo1', 'misplacedimages110044', 'misplacedimages120024']
Enter name for this set of runs:foo1


## Run the cell below to begin analyzing.

Check the output; it will update as images are analyzed.

To stop the analysis, press the interrupt kernel/stop button at the very top. You can restart the analysis by simply running the cell below again. The console output will be cleared, but not to worry, it's all in the log file.

In [3]:
import os
import time
import datetime
import shutil
import posixpath
import sys
# from breadboard import BreadboardClient
import warnings
import pandas as pd
from numpy import isnan
from logparsing import translate_stringy_list

"""import analysis function, which accepts filepath and optionally previous settings, from analysis_wrappers.py, and 
returns (analysis_dict, settings_dict)."""
if analysis_type == 'fake analysis':
    analysis_function = analysis_wrappers.fake_analysis1
    analyzed_var_names = ['fake analysis 1', 'fake analysis 2']
elif analysis_type == 'fake analysis 2':
    analysis_function = analysis_wrappers.fake_analysis2
    analyzed_var_names = ['fake analysis 3', 'fake analysis 4']
elif analysis_type == 'ycam':
    analyzed_var_names = ['bareNcntAverageMarqueeBoxValues', 
                          'COMX', 'COMY'] #TODO set MATLAB analyzed_var_names properly
    analysis_function = analysis_wrappers.ycam_analysis
elif analysis_type == 'zcam_dual_imaging':
    analyzed_var_names = ['K_bareNcntAverageMarqueeBoxValues', 'Na_bareNcntAverageMarqueeBoxValues'
                          'Na_COMX', 'Na_COMY'] #TODO set MATLAB analyzed_var_names properly
    analysis_function = analysis_wrappers.dual_imaging_analysis

warnings.filterwarnings(
    "ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings(
    "ignore", "Could not find appropriate MS Visual C Runtime")
# bc = BreadboardClient(config_path='API_CONFIG_fermi1.json')

def analysis_to_imagelog(image_log_df, idx, outputlog_filename, previous_settings = None,
                        output_previous_settings = True):
    image_filename = image_log_df.loc[idx, 'filename0']
    abs_image_path = os.path.join(os.path.join(os.getcwd(), measurement_dir), image_filename)
    print(abs_image_path)
    settings = None
    if analysis_type == 'fake analysis' or analysis_type == 'fake analysis 2':
        analysis_dict, settings = analysis_function(image_filename)
    elif previous_settings is None:
        analysis_dict, settings = analysis_function(abs_image_path)#no marquee box yet
    else:
        analysis_dict, settings = analysis_function(abs_image_path, previous_settings)
    if not output_previous_settings:
        settings = None #forces user to select new marquee box for each shot
    image_log_df = pd.read_csv(outputlog_filename) #get updated copy of log to prevent overwriting
    print('{file} analyzing: '.format(file=image_filename))
    for key in analyzed_var_names:
        print(key, analysis_dict[key])
        image_log_df.loc[idx,key] = analysis_dict[key]
    image_log_df.to_csv(outputlog_filename, index = False)
#     print('{file} analyzed: '.format(file=image_filename) + str(analysis_dict))
    return image_log_df, settings

def main():
    # Global settings
    # bc = BreadboardClient(config_path='API_CONFIG.json') # enter your path to the API_config
    refresh_time = 1  # seconds
    global watchfile
    previous_settings = None
    print("\n\n Watching this file for changes: " + watchfile + "\n\n")
    # Main Loop
    while True:
        if not os.path.exists(watchfile):
            time.sleep(refresh_time)
            continue
        else:
            image_log_df = pd.read_csv(watchfile)

        for var_name in analyzed_var_names:
            if var_name not in image_log_df.columns: #start new analysis type
#             print('{analysis} analysis started'.format(analysis = analysis_type))
                image_log_df, previous_settings = analysis_to_imagelog(image_log_df, 0, watchfile, previous_settings)
                if 'analyzed_variables' not in image_log_df.columns:
                    image_log_df.loc[0,'analyzed_variables'] = str(analyzed_var_names)
                    image_log_df.to_csv(watchfile, index = False)
                else:
                    old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[0,'analyzed_variables']))
                    updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
                    image_log_df.loc[0,'analyzed_variables'] = str(updated_analyzed_variables)
                    image_log_df.to_csv(watchfile, index = False)
            else: #queue un-analyzed images
                for idx in reversed(range(len(image_log_df.index))): #start from most recent image first
                    if isnan(image_log_df.loc[idx, var_name]):
                        image_log_df, previous_settings = analysis_to_imagelog(image_log_df, idx, watchfile, previous_settings)
                        if isinstance(image_log_df.loc[idx,'analyzed_variables'],float):
                            image_log_df.loc[idx,'analyzed_variables'] = str(analyzed_var_names)
                            image_log_df.to_csv(watchfile, index = False)
                        else:
                            old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[idx,'analyzed_variables']))
                            updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
                            image_log_df.loc[idx,'analyzed_variables'] = str(updated_analyzed_variables)
                            image_log_df.to_csv(watchfile, index = False)

            # # Write to Breadboard
            # try:
            #     resp = bc.post_images(
            #                 image_names = os.path.splitext(output_filename)[0],
            #                 filepath = output_filepath,
            #                 image_times = [dt],
            #                 auto_time = False # Add more information here
            #                 )
            #     if resp.status_code!=200:
            #         print(resp.text)
            # except: pass
            # names_old = names
#             length_old = len(image_log_df.index)
        time.sleep(refresh_time)

main()



 Watching this file for changes: 08\200806\foo1\image_log.csv


C:\Users\Alex\Dropbox (MIT)\lab side projects\control software\enrico\08\200806\foo1\image00.csv
image00.csv analyzing: 
fake analysis 1 8
fake analysis 2 27
C:\Users\Alex\Dropbox (MIT)\lab side projects\control software\enrico\08\200806\foo1\image10_120036.csv
image10_120036.csv analyzing: 
fake analysis 1 24
fake analysis 2 14


KeyboardInterrupt: 